# Introduction
What is the objective of this notebook? </br>
The objective of this notebook is to create a scipt which can make a repository of recipes and their ingredients. </br>

Below i will import the necessary libraries

In [1]:
import pandas as pd
import os
from datetime import datetime
from tqdm import tqdm
import regex as re
import time
import numpy as np
import random
import pyperclip as py
import openai 
import os
import pandas as pd
# from    dotenv import load_dotenv
# load_dotenv()
# openai.api_type     = os.getenv('OPENAI_TYPE')
# openai.api_base     = os.getenv('OPENAI_BASE')
# openai.api_version  = os.getenv('OPENAI_VERSION')
# openai.api_key      = os.getenv('OPENAI_KEY')
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse, urljoin
from tqdm import tqdm

import helper as helper

In [2]:
# Load the recipes data 
recipes = pd.read_csv('recipes.csv')
# Load the ingredients data
ingredients = pd.read_csv('ingredients_df.csv')
# Load the pantry data
pantry = pd.read_csv('pantry.csv')
# Load the conversion data
conversion = pd.read_csv('conversion.csv')


In [4]:
def urls_on_same_domain(url, max_urls=1000):
    """
    Counts the number of unique URLs on the same domain as the given URL,
    iterating through links until no more are found or a max limit is reached.
    The progress is displayed using tqdm.

    Args:
    url (str): The starting URL to fetch and analyze.
    max_urls (int): Maximum number of URLs to fetch.

    Returns:
    int: The total number of unique URLs found on the same domain.
    """
    visited_urls = set()
    urls_to_visit = set([url])
    domain = urlparse(url).netloc

    with tqdm(total=max_urls, desc=f"Analyzing URLs currently found {len(visited_urls)}, and have {len(urls_to_visit)} to go", unit="url") as pbar:
        while len(urls_to_visit) != 0 and len(urls_to_visit) < max_urls:
            current_url = urls_to_visit.pop()
            try:
                response = requests.get(current_url)
                response.raise_for_status()

                soup = BeautifulSoup(response.content, 'html.parser')
                anchors = soup.find_all('a', href=True)

                for anchor in anchors:
                    href = anchor['href']
                    full_url = urljoin(current_url, href)
                    href_domain = urlparse(full_url).netloc

                    if href_domain == domain and full_url not in visited_urls and '#comment' not in full_url and '#respond' not in full_url:
                        urls_to_visit.add(full_url)

                visited_urls.add(current_url)
                pbar.update(1)
                pbar.set_description(f"Analyzing URLs currently found {len(visited_urls)}, and have {len(urls_to_visit)} to go")
            except requests.RequestException:
                # Handle exceptions for requests
                continue

    return visited_urls, urls_to_visit

def get_text_from_url(url):
    """
    Fetches the content from a given URL and returns the text separated by lines.
    If the URL gets redirected more than twice, returns 'redirect link'.

    Args:
    url (str): The URL from which to fetch the content.

    Returns:
    str: A string containing the text from the URL, separated by lines, or 'redirect link'.
    """
    # Send a GET request to the URL with redirection allowed
    response = requests.get(url, allow_redirects=True)
    
    # Check the number of redirects
    if len(response.history) > 2:
        return 'redirect link'
    
    # Raise an exception if the request was unsuccessful
    response.raise_for_status()

    # Parse the content of the request with BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    # Extract and return the text, separated by lines
    return '\n'.join(soup.stripped_strings)

def ingredients_cleaner(text):
    try:
        text = text.split('Ingredienser')[1]
        text = text.split('Udskriv')[0]
        text = text.strip()
    except:
        text = ''
    if 'Archive' in text:
        text = ''
    return text

def set_individual_price(row):
    amount = row['amounts']
    purchase_size = row['purchase_size']
    price = row['price']
    input_units = row['units']
    purchase_unit = row['purchase_unit']
    if len(conversion[(conversion.units == input_units) & (conversion.purchase_unit == purchase_unit)] ) > 0:
        faktor = conversion[(conversion.units == input_units) & (conversion.purchase_unit == purchase_unit)].rate.values[0]
    else:
        return 0

    if purchase_size == 0:
        return 0
    else:
        if input_units == purchase_unit:
            y = np.ceil(amount / purchase_size) * price if amount > purchase_size else price
        else:
            if input_units == 'g' and purchase_unit == 'stk':
                y = price
            else:
                # we need to make it so that it looks in the conversion table and then returns the 
                y = np.ceil(amount * faktor / purchase_size) * price  if amount > purchase_size else price
    
    # lets also round the product of faktor by purchase_size and amount
    z = np.ceil(amount / purchase_size) * faktor if amount > purchase_size else faktor * amount
    
    return y

def find_dinner(test_text, test_url):
    segment = test_text.split(test_url)[1]
    if 'Aftensmad' in segment:
        if 'Tilbehør Aftensmad' in segment:
            return False
        return True
    else:
        return False

def find_vegeterian(test_text, test_url):
    segment = test_text.split(test_url)[1]
    if 'Vegetar' in segment:
        return True
    else:
        return False

def find_addon(test_text, test_url):
    segment = test_text.split(test_url)[1]
    if 'Tilbehør Aftensmad' in segment:
        return True
    else:
        return False

def ing_extract_measurement(line, measurements):
    """This function extracts the measurement from a line of text, from the ingredients_df

    Args:
        line (str): ingredients line
        measurements (list): list of measurements

    Returns:
        str: measurement
    """
    local_measurement = ''

    for measurement in measurements:
        if f'{measurement} ' in line:
            return measurement
        
    return local_measurement

def implied_terms(ingredient):
    if 'øko' in ingredient:
        ingredient = ingredient.replace('øko', '')

    return ingredient

def extract_measurement(line, measurements):
    local_measurement = ''

    for measurement in measurements:
        if f' {measurement} ' in line:
            return measurement
        
    return local_measurement

def extract_amount(line):
    pattern = '^\d+'
    match = re.findall(pattern, line)
    if len(match) > 0:
        return match[0]
    else:
        return 0

def extract_ingredient(line, measurement):
    """This function extracts the ingredient from a line of text

    Args:
        line (str): a line of text containg an ingredient, amount and measurement
        measurement (list): a list of measurements

    Returns:
        amount: the amount of the ingredient
        measurement: the measurement of the ingredient
        new_line: the ingredient
    """
    amount = extract_amount(line)
    local_measurement = extract_measurement(line, measurement)
    if local_measurement != '':
        new_line = line.replace(amount, '')
        new_line = new_line.replace(f' {local_measurement} ', '')
        new_line = new_line.strip()
    else:
        new_line = line.replace(amount, '')
        new_line = new_line.strip()

    # now lets clean up after the measurement
    if ',' in amount:
        amount = amount.replace(',', '.')
        new_line = new_line.replace(amount, '')
        new_line = new_line.strip()
    else:
        new_line = new_line.replace(amount, '')
        new_line = new_line.strip()

    # now lets remove the implied terms
    new_line = implied_terms(new_line)
    new_line = new_line.strip()

    return amount, local_measurement, new_line

def ingredients_setter(ingredients_text, url, measurements, pantry):
    """This function takes a string of ingredients and sets them in the pantry"""
    ingredients_text = ingredients_cleaner(ingredients_text)    

    for line in ingredients_text.splitlines():
        amount, local_measurement, new_line = extract_ingredient(line, measurements)
        pantry.loc[len(pantry)] = [url, amount, local_measurement, new_line]

    return pantry

def get_measurements(conversion):
    """This function returns a list of all the measurements that we have in the conversion table

    Args:
        conversion (DataFrame): the conversion dataframe 

    Returns:
        list: a list of all the measurements that we have in the conversion table
    """
    # weird measurements
    measurements = ['drys', 'dryp', 'skiver', 'skive', 'stængel', 'stængler', 'ark', 'dåser', 'dåse', 'pakke', 'pakker', 'håndfulde', 'håndfuld']

    # now lets get all the measurements from the conversion table, and add them to the measurements list
    for measurement in conversion.units.unique():
        measurements.append(measurement)

    return measurements

measurements = get_measurements(conversion)


In [6]:

def urls_on_same_domain(url, max_urls=1000):
    """
    Counts the number of unique URLs on the same domain as the given URL,
    iterating through links until no more are found or a max limit is reached.
    The progress is displayed using tqdm.

    Args:
    url (str): The starting URL to fetch and analyze.
    max_urls (int): Maximum number of URLs to fetch.

    Returns:
    int: The total number of unique URLs found on the same domain.
    """
    visited_urls = set()
    urls_to_visit = set([url])
    domain = urlparse(url).netloc

    # Common user-agent string of a web browser
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }

    with tqdm(total=max_urls, desc=f"Analyzing URLs currently found {len(visited_urls)}, and have {len(urls_to_visit)} to go", unit="url") as pbar:
        while len(urls_to_visit) != 0 and len(visited_urls) < max_urls:
            current_url = urls_to_visit.pop()
            try:
                response = requests.get(current_url, headers=headers)
                response.raise_for_status()

                soup = BeautifulSoup(response.content, 'html.parser')
                anchors = soup.find_all('a', href=True)

                for anchor in anchors:
                    href = anchor['href']
                    full_url = urljoin(current_url, href)
                    href_domain = urlparse(full_url).netloc

                    if href_domain == domain and full_url not in visited_urls and '#comment' not in full_url and '#respond' not in full_url:
                        urls_to_visit.add(full_url)

                visited_urls.add(current_url)
                pbar.update(1)
                pbar.set_description(f"Analyzing URLs currently found {len(visited_urls)}, and have {len(urls_to_visit)} to go")
            except requests.RequestException:
                # Handle exceptions for requests
                continue

    return len(visited_urls)

def get_text_from_url(url):
    """
    Fetches the content from a given URL and returns the text separated by lines.
    If the URL gets redirected more than twice, returns 'redirect link'.

    Args:
    url (str): The URL from which to fetch the content.

    Returns:
    str: A string containing the text from the URL, separated by lines, or 'redirect link'.
    """
    # Common user-agent string of a web browser
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }

    # Send a GET request to the URL with the user-agent header and redirection allowed
    response = requests.get(url, headers=headers, allow_redirects=True)
    
    # Check the number of redirects
    if len(response.history) > 2:
        return 'redirect link'
    
    # Raise an exception if the request was unsuccessful
    response.raise_for_status()

    # Parse the content of the request with BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    # Extract and return the text, separated by lines
    return '\n'.join(soup.stripped_strings)

url = 'https://vegetariskhverdag.dk/opskrifter/'

#test = get_text_from_url(url)

In [7]:
def get_measurements(conversion):
    """This function returns a list of all the measurements that we have in the conversion table

    Args:
        conversion (DataFrame): the conversion dataframe 

    Returns:
        list: a list of all the measurements that we have in the conversion table
    """
    # weird measurements
    measurements = ['drys', 'dryp', 'skiver', 'skive', 'stængel', 'stængler', 'ark', 'dåser', 'dåse', 'pakke', 'pakker', 'håndfulde', 'håndfuld']

    # now lets get all the measurements from the conversion table, and add them to the measurements list
    for measurement in conversion.units.unique():
        measurements.append(measurement)

    return measurements

measurements = get_measurements(conversion)

get_measurements(conversion)

removeable = ['drys', 'dryp', 'skiver', 'skive', 'stængel', 'stængler', 'ark', 'dåser', 'dåse', 'pakke', 'pakker', 'håndfulde', 'bakke', 'bakker']
# removeable = ['håndfulde', 'håndfuld']

for ingredient in ingredients.ingredients.unique():
    local_measurement = ing_extract_measurement(ingredient, measurements)
    new_ingredient = ''
    if local_measurement in removeable:
        new_ingredient = ingredient.replace(f'{local_measurement} ', '')
        new_ingredient = new_ingredient.strip()

        if new_ingredient != '':
            print(ingredient, new_ingredient, local_measurement)
            ingredients.loc[ingredients.ingredients == ingredient, 'ingredients'] = new_ingredient
            # now lets make sure that we set the unit to grams
            # pantry.loc[pantry.ingredients == ingredient, 'units'] = 'g'
            # now lets multiply the amount by 25 grams
            # pantry.loc[pantry.ingredients == ingredient, 'amounts'] = pantry.loc[pantry.ingredients == new_ingredient, 'amounts'] * 25
            pantry.loc[pantry.ingredients == ingredient, 'ingredients'] = new_ingredient

        else:
            print('error',ingredient)

        

håndfulde baby spinat baby spinat håndfulde
håndfulde isterninger isterninger håndfulde


In [16]:
text = open('dovne_vegetar.txt', 'r').read()
text = text.splitlines()
# now lets remove any item which contains 'reply-title'
text = [item for item in text if 'reply-title' not in item]
text = [item for item in text if '/vare/' not in item]
text = [item for item in text if '/wp-content/' not in item]

df = pd.DataFrame(text, columns=['url'])
df['text']  = ''

for index, row in tqdm(df.iterrows(), total=df.shape[0]):
    df.loc[index, 'text'] = get_text_from_url(row['url'])

100%|██████████| 407/407 [04:52<00:00,  1.39it/s]


In [30]:
# now lets test if the texts contain '\nIngredienser\n'
df['has_ingredienser'] = df.text.apply(lambda x: True if '\nIngredienser\n' in x else False)

In [34]:
df = df[df.has_ingredienser == True].copy()

In [37]:
df.reset_index(inplace=True, drop=True)

In [55]:
sample = df.text.sample(1).values[0]
sample = sample.split('\nIngredienser')[1]
sample = sample.split('portioner\n')[1]
sample

'2 dl røde ris (eller andre ris)\n1 god håndfuld cashewnødder\nGrøntsager efter eget valg (her: 1 rødløg, 2 røde snackpeber & 1/2 spidskålshoved)\n1 spsk revet ingefær\n1-2 fed hvidløg, fintrevet\n3 spsk grøntsagsfond (eller vand + et nip salt)\n2 tsk majsstivelse (maizena)\n2 spsk soja\n1 spsk lys sirup (eller honning/sukker/agave)\n2 spsk riseddike eller anden lys eddike\n1-1,5 spsk ristet sesamolie\n1-2 tsk chili flakes\nSådan gør du\nSæt risene over efter instruktionerne på pakken.\nRist cashewnødderne gyldne.\nSkær grøntsagerne ud i passende stykker. Jeg skar løget i skiver og de andre grøntsager i grove stykker. Riv ingefær og hvidløg fint.\nBland fond/vand (kold) og maizena sammen i en skål. Tilsæt soja, sirup, eddike og sesamolie.\nSteg grøntsagerne ved høj varme, så de får godt med farve. Du kan evt stege dem i omgange. Tilsæt ingefær og hvidløg til sidst, så det ikke brænder på. Steg i 1-2 minutter og hæld så din sovs over. Lad simre i 1 minut og smag til med evt mere soja/si

In [56]:
def extract_measurement(line, measurements):
    local_measurement = ''

    for measurement in measurements:
        if f' {measurement} ' in line:
            return measurement
        
    return local_measurement

def extract_amount(line):
    pattern = '^\d+'
    match = re.findall(pattern, line)
    if len(match) > 0:
        return match[0]
    else:
        return 0

def extract_ingredient(line, measurement):
    """This function extracts the ingredient from a line of text

    Args:
        line (str): a line of text containg an ingredient, amount and measurement
        measurement (list): a list of measurements

    Returns:
        amount: the amount of the ingredient
        measurement: the measurement of the ingredient
        new_line: the ingredient
    """
    amount = extract_amount(line)
    local_measurement = extract_measurement(line, measurement)
    if local_measurement != '':
        new_line = line.replace(amount, '')
        new_line = new_line.replace(f' {local_measurement} ', '')
        new_line = new_line.strip()
    else:
        new_line = line.replace(amount, '')
        new_line = new_line.strip()

    # now lets clean up after the measurement
    if ',' in amount:
        amount = amount.replace(',', '.')
        new_line = new_line.replace(amount, '')
        new_line = new_line.strip()
    else:
        new_line = new_line.replace(amount, '')
        new_line = new_line.strip()

    # now lets remove the implied terms
    new_line = implied_terms(new_line)
    new_line = new_line.strip()

    return amount, local_measurement, new_line

def ingredients_setter(ingredients_text, url, measurements, pantry):
    """This function takes a string of ingredients and sets them in the pantry"""
    ingredients_text = ingredients_cleaner(ingredients_text)    

    for line in ingredients_text.splitlines():
        amount, local_measurement, new_line = extract_ingredient(line, measurements)
        pantry.loc[len(pantry)] = [url, amount, local_measurement, new_line]

    return pantry

def get_measurements(conversion):
    """This function returns a list of all the measurements that we have in the conversion table

    Args:
        conversion (DataFrame): the conversion dataframe 

    Returns:
        list: a list of all the measurements that we have in the conversion table
    """
    # weird measurements
    measurements = ['drys', 'dryp', 'skiver', 'skive', 'stængel', 'stængler', 'ark', 'dåser', 'dåse', 'pakke', 'pakker', 'håndfulde', 'håndfuld']

    # now lets get all the measurements from the conversion table, and add them to the measurements list
    for measurement in conversion.units.unique():
        measurements.append(measurement)

    return measurements

measurements = get_measurements(conversion)

for line in sample.splitlines():
    amount, local_measurement, new_line = extract_ingredient(line, measurements)
    print(amount, local_measurement, new_line)

2 dl røde ris (eller andre ris)
1 håndfuld godcashewnødder


TypeError: replace() argument 1 must be str, not int

In [59]:
df.to_csv('dovne_vegetar.csv', index=False)